<a href="https://colab.research.google.com/github/insaiyancvk/Computer-Vision/blob/main/Image_colorize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>